# Spam Classification

I was first exposed to this exercise in [Andrew Ng's Intro to Machine Learning class on Coursera](https://www.coursera.org/learn/machine-learning). I revisited it through end-of-chapter-3 exercise in [Aurélien Géron's Machine Learning Handbook](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/) in order to extend my understanding of the concepts as well as the relevant python tools. I am using Aurélien's code on [github](https://github.com/ageron/handson-ml2) with some tweaks for 20050311_spam_2 and 20030228_hard_ham data from [spamassassin corpus](http://spamassassin.apache.org/old/publiccorpus/).

In [72]:
# setup environment
import os
import urllib
import tarfile
import email
import email.policy
import nltk
import urlextract
import numpy as np
import re


from collections import Counter
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score

In [2]:
# get email data from spamAssassin public corpus
ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
SPAM_URL = "".join([ROOT, "20050311_spam_2.tar.bz2"])
HAM_URL = "".join([ROOT, "20030228_hard_ham.tar.bz2"])
DATA_PATH = os.path.join("data", "spamClf")

# define function to pull and extract data
def getSpamData(dataUrl=SPAM_URL, dataPath=DATA_PATH):
    '''pull and extract spamAssasin email data'''
    if not os.path.isdir(dataPath):
        os.makedirs(dataPath)
    for f, url in [["spam_2.tar.bz2", SPAM_URL], ["hard_ham.tar.bz2", HAM_URL]]:
        path = os.path.join(dataPath, f)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
            tar_bz2_f = tarfile.open(path)
            tar_bz2_f.extractall(path=DATA_PATH)
            tar_bz2_f.close()

In [3]:
getSpamData(dataUrl=SPAM_URL, dataPath=DATA_PATH) # call function

In [11]:
# load data (emails)
SPAM_DIR = os.path.join(DATA_PATH, "spam_2")
HAM_DIR = os.path.join(DATA_PATH, "hard_ham")

# filenames are 38 characters
spam_files = [f for f in sorted(os.listdir(SPAM_DIR)) if len(f) >= 35]
ham_files = [f for f in sorted(os.listdir(HAM_DIR)) if len(f) >= 35]

In [13]:
# number of pulled spam and ham files
print(f"Number of spam files: {len(spam_files)}")
print(f"Number of ham files: {len(ham_files)}")

Number of spam files: 1396
Number of ham files: 250


This (spam_2 and hard_ham) set of data a considerably larger ratio (1396/250) of spam file over ham files. The "spam" and "easy_ham" sets had this ratio at 500/2500.

In [14]:
# function to load emails (create email parser instance)
def loadEmail(isSpam, file, spamPath=DATA_PATH):
    if isSpam:
        folder = "spam_2"
    else:
        folder = "hard_ham"
    with open(os.path.join(spamPath, folder, file), "rb") as f:
        # parser API used (vs. feedparser) as emails in files (not livefeed)
        # create BytesParser instance
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [15]:
spam_emails = [loadEmail(isSpam=True, file=f) for f in spam_files]
ham_emails = [loadEmail(isSpam=False, file=f) for f in ham_files]

In [16]:
# get email structures
def getEmailStructure(email):
    '''get structure of email'''
    if isinstance(email, str):
        return email
    payload = email.get_payload() # list of multipart message objects
    if isinstance(payload, list):
        return "multipart({})".format(", ".join([
            getEmailStructure(subEmail) for subEmail in payload
        ]))
    else:
        return email.get_content_type() # email's content type

In [17]:
# split data into training and test sets

# list of email objects
X = np.array(spam_emails + ham_emails, dtype=object)

# target labels
y = np.array([1] * len(spam_emails) + [0] * len(ham_emails))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [24]:
# print first ham sample from X_train 
print(X_train[y_train==0][3].get_content().strip())

I am trying to secure three of four virtual hostnames on our Apache server.
We are not taking credit card orders or user's personal information, but are
merely hoping to secure email and calendar web transactions for our users.
We are not running any secure applications on the root host.

I have been testing this week with CA, client, and host certificate
requests, certificates, and keys, and think I have a fairly good beginner's
grasp of the commands and command line options.


My questions are:

1.  Is it necessary to create a CA certificate for each of the secure
virtual hosts, or can one CA certificate for the root be used to sign each
of the keys for all three common names we are trying to secure?

2.  Even though the root host is not conducting secure transactions, am I
correct in configuring the server with a CACertificateFile in the main body
of httpsd.conf and then setting the CACertificateFile for each virtual host
in the <Virtual . . .> section of httpsd.conf?  This sort of 

In [47]:
# preprocess ...

# function to converst email body into bag of words
def html2text(html):
    soup = BeautifulSoup(html, features="lxml")
    text = soup.get_text() # remove html markups
    
    if soup.head: soup.head.decompose() # remove headers
    for a in soup.find_all("a"):
        a.replace_with(" HYPERLINK ") # convert all <a> tags with text HYPERLINK
    for s in soup(["script", "style"]):
        s.decompose() # remove tags
    text = ' '.join(soup.stripped_strings) # retrieve tag contents
    return text

In [48]:
# check html2text works - print spam html
htmlSpamEmails = [email for email in X_train[y_train==1]
                 if getEmailStructure(email) == "text/html"]

sampleHtmlSpam = htmlSpamEmails[10]
# first 500 characters
print(sampleHtmlSpam.get_content().strip()[:500], "...")

<html><body bgColor="#CCCCCC" topmargin=1 onMouseOver="window.status=''; return true" oncontextmenu="return false" ondragstart="return false" onselectstart="return false">
<div align="center">Hello, jm@netnoteinc.com<BR><BR></div><div align="center"></div><p align="center"><b><font face="Arial" size="4">Human Growth Hormone Therapy</font></b></p>
<p align="center"><b><font face="Arial" size="4">Lose weight while building lean muscle mass<br>and reversing the ravages of aging all at once.</font>< ...


In [49]:
# check html2text works - print spam text (using html2text function)
print(html2text(sampleHtmlSpam.get_content())[:500], "...")

Hello, jm@netnoteinc.com Human Growth Hormone Therapy Lose weight while building lean muscle mass and reversing the ravages of aging all at once. Remarkable discoveries about Human Growth Hormones ( HGH ) are changing the way we think about aging and weight loss. Lose Weight Build Muscle Tone Reverse Aging Increased Libido Duration Of Penile Erection Healthier Bones Improved Memory Improved skin New Hair Growth Wrinkle Disappearance HYPERLINK You are receiving this email as a subscr iber to the  ...


In [50]:
# check html2text works - print spam text (using html2text function)
print(html2text(sampleHtmlSpam.get_content())[:500], "...")

Hello, jm@netnoteinc.com Human Growth Hormone Therapy Lose weight while building lean muscle mass and reversing the ravages of aging all at once. Remarkable discoveries about Human Growth Hormones ( HGH ) are changing the way we think about aging and weight loss. Lose Weight Build Muscle Tone Reverse Aging Increased Libido Duration Of Penile Erection Healthier Bones Improved Memory Improved skin New Hair Growth Wrinkle Disappearance HYPERLINK You are receiving this email as a subscr iber to the  ...


In [51]:
# preprocess ...
# function to convert any content to plain text
def email2text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ("text/plain", "text/html"):
            continue
        try:
            content = part.get_content()
        except:
            content = str(part.get_payload())
        if ctype == "text/plain":
            return content
        else:
            html = content
    if html:
        return html2text(html)

In [52]:
print(email2text(sampleHtmlSpam)[:100], "...")

Hello, jm@netnoteinc.com Human Growth Hormone Therapy Lose weight while building lean muscle mass an ...


In [53]:
# preprocess ...
stemmer = nltk.PorterStemmer() # initialize stemmer
url_extractor = urlextract.URLExtract() # initialize url extractor

In [54]:
# preprocess ...
# class to convert emails to word counters
#nltk.download("stopwords")

class Emails2WordCounts(BaseEstimator, TransformerMixin):
    '''convert emails to word counters'''
    def __init__(self, strip_headers=True, replace_urls=True,
                 replace_numbers=True, remove_punctuation=True,
                 stemming=True, lowercase=True,
                remove_stopwords=True):
        self.strip_headers = strip_headers
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.remove_punctuation = remove_punctuation
        self.stemming = stemming
        self.lowercase = lowercase
        self.remove_stopwords=remove_stopwords
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed = []
        for email in X:
            text = email2text(email) or ''
            if self.lowercase:
                text = text.lower()
            if self.replace_urls:
                urls = list(set(url_extractor.find_urls(text)))
                urls.sort(key=lambda url: len(url), reverse=True)
            for url in urls:
                text = text.replace(url, "URL")
            if self.replace_numbers:
                regx = re.compile(r'\d+(?:\.\d*(?:[eE]\d+))?')
                text = regx.sub(string=text, repl="NUMBER")
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text, flags=re.M)
                regx = re.compile(r"([^\w\s]+)|([_-]+)")
                text = regx.sub(string=text, repl=" ")
            if self.remove_stopwords:
                words = text.split()
                keepWords = [word for word in words if 
                             word not in stopwords.words('english')] 
            word_counts = Counter(keepWords)
            if self.stemming:
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [55]:
# check it for a few email samples
X_few = X_train[50:53]
X_few_wc = Emails2WordCounts().fit_transform(X_few)
X_few_wc

array([Counter({'number': 28, 'us': 11, 'cd': 7, 'address': 7, 'name': 7, 'busi': 6, 'data': 6, 'compani': 5, 'fax': 5, 'card': 5, 'order': 5, 'usa': 4, 'databas': 4, 'domain': 4, 'ship': 4, 'mail': 4, 'remov': 4, 'code': 3, 'complet': 3, 'registr': 3, 'credit': 3, 'form': 3, 'product': 2, 'inform': 2, 'everi': 2, 'one': 2, 'contain': 2, 'yellow': 2, 'page': 2, 'fresh': 2, 'fill': 2, 'creat': 2, 'web': 2, 'site': 2, 'format': 2, 'send': 2, 'email': 2, 'import': 2, 'free': 2, 'print': 2, 'datanetwork': 2, 'e': 2, 'pleas': 2, 'citi': 2, 'state': 2, 'postal': 2, 'date': 2, 'subject': 2, 'offer': 1, 'let': 1, 'develop': 1, 'custom': 1, 'base': 1, 'time': 1, 'compil': 1, 'internet': 1, 'singl': 1, 'million': 1, 'sort': 1, 'categori': 1, 'sic': 1, 'accuraci': 1, 'sold': 1, 'thousand': 1, 'complaint': 1, 'commit': 1, 'deliv': 1, 'best': 1, 'qualiti': 1, 'highli': 1, 'reward': 1, 'book': 1, 'relat': 1, 'select': 1, 'within': 1, 'area': 1, 'across': 1, 'whole': 1, 'put': 1, 'html': 1, 'think': 

In [56]:
# preprocess ...
# build list (orderd) of most common words and convert it to vectors
class WordCounts2Vector(BaseEstimator, TransformerMixin):
    '''convert word counts to vectors'''
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X: # counter dictionary object
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.most_common_ = most_common
        self.vocabulary_ = {word: index + 1 for index, (word, count) in 
                            enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), 
                          shape=(len(X), self.vocabulary_size + 1))

In [57]:
vocab_transformer = WordCounts2Vector(vocabulary_size=10)
X_few_vectors = vocab_transformer.fit_transform(X_few_wc)
X_few_vectors

<3x11 sparse matrix of type '<class 'numpy.longlong'>'
	with 19 stored elements in Compressed Sparse Row format>

In [58]:
X_few_vectors.toarray()
# first column represents words in that row(email) that 
#     do not appear in the vocabulary
# the rest of the columns represent the times the vocabulary
#     words appear in the email
# the vocabulary is obtained using the vocabulary_ 
#     attribute of the class

array([[226,  28,  11,   0,   6,   7,   4,   7,   7,   0,   6],
       [ 42,   0,   0,   0,   0,   0,   3,   0,   0,   0,   0],
       [312,  14,   1,  59,   2,   1,   1,   0,   0,   7,   0]],
      dtype=int64)

In [59]:
vocab_transformer.vocabulary_

{'number': 1,
 'us': 2,
 'hyperlink': 3,
 'busi': 4,
 'address': 5,
 'mail': 6,
 'cd': 7,
 'name': 8,
 'electron': 9,
 'data': 10}

In [60]:
# training pipeline
preprocess_pipeline = Pipeline([
    ("email2wordcounts", Emails2WordCounts()),
    ("wordcounts2vector", WordCounts2Vector()),
])

%time X_train_transformed = preprocess_pipeline.fit_transform(X_train)

CPU times: user 2min 1s, sys: 25.2 s, total: 2min 26s
Wall time: 3min 49s


In [73]:
log_clf = LogisticRegression(solver="lbfgs", 
                             max_iter=1000, random_state=42)
score = cross_val_score(log_clf, X_train_transformed,
                        y_train, cv=3, verbose=3)

print(f"\nMean cross validation score: {score.mean():.4f}")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  ................................................................
[CV] .................................... , score=0.954, total=   0.2s
[CV]  ................................................................
[CV] .................................... , score=0.952, total=   0.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s


[CV] .................................... , score=0.952, total=   0.3s

Mean cross validation score: 0.9529


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.8s finished


In [62]:
%time X_test_transformed = preprocess_pipeline.transform(X_test)

log_clf = LogisticRegression(solver="lbfgs", max_iter=1000,
                             random_state=42)
log_clf.fit(X_train_transformed, y_train)

y_pred = log_clf.predict(X_test_transformed)

print("\nPrecision: {:.2f}%".format(
    100 * precision_score(y_test, y_pred)))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_pred)))

CPU times: user 27.2 s, sys: 5.51 s, total: 32.7 s
Wall time: 50.4 s

Precision: 97.60%
Recall: 98.62%


The logistic regression model gives reasonably high Precision and Recall scores on the test set. Lets check the performance using sklearn SVM (SVC) with GridSearch. 

In [82]:
# estimator
svm_clf = SVC()

# grid search parameters
param_grid = {
    'C': np.logspace(-1, 2, 10), # 0.1-100
    'gamma': np.logspace(-1, 1, 10), # 0.1-10
    'kernel': ["linear", "rbf"]
}

# perform grid
grid_search = GridSearchCV(svm_clf, param_grid, cv=3, verbose=3)

In [83]:
# fit best model
%time grid_search.fit(X_train_transformed, y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.957, total=   0.2s
[CV] C=0.1, gamma=0.1, kernel=linear .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.948, total=   0.2s
[CV] C=0.1, gamma=0.1, kernel=linear .................................
[CV] ..... C=0.1, gamma=0.1, kernel=linear, score=0.947, total=   0.2s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.841, total=   0.5s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.841, total=   0.5s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.842, total=   0.5s
[CV] C=0.1, gamma=0.16681005372000587, kernel=linear .................
[CV]  C=0.1, gamma=0.16681005372000587, kernel=linear, score=0.957, total=   0.2s
[CV] C=0.1, gamma=0.16681005372000587, kernel=linear .................
[CV]  C=0.1, gamma=0.16681005372000587, kernel=linear, score=0.948, total=   0.2s
[CV] C=0.1, gamma=0.16681005372000587, kernel=linear .................
[CV]  C=0.1, gamma=0.16681005372000587, kernel=linear, score=0.947, total=   0.2s
[CV] C=0.1, gamma=0.16681005372000587, kernel=rbf ....................
[CV]  C=0.1, gamma=0.16681005372000587, kernel=rbf, score=0.841, total=   0.6s
[CV] C=0.1, gamma=0.1668100537200058

...

[CV]  C=46.41588833612777, gamma=5.994842503189409, kernel=linear, score=0.925, total=   0.2s
[CV] C=46.41588833612777, gamma=5.994842503189409, kernel=linear .....
[CV]  C=46.41588833612777, gamma=5.994842503189409, kernel=linear, score=0.934, total=   0.2s
[CV] C=46.41588833612777, gamma=5.994842503189409, kernel=rbf ........
[CV]  C=46.41588833612777, gamma=5.994842503189409, kernel=rbf, score=0.850, total=   0.6s
[CV] C=46.41588833612777, gamma=5.994842503189409, kernel=rbf ........
[CV]  C=46.41588833612777, gamma=5.994842503189409, kernel=rbf, score=0.850, total=   0.6s
[CV] C=46.41588833612777, gamma=5.994842503189409, kernel=rbf ........
[CV]  C=46.41588833612777, gamma=5.994842503189409, kernel=rbf, score=0.845, total=   0.5s
[CV] C=46.41588833612777, gamma=10.0, kernel=linear ..................
[CV]  C=46.41588833612777, gamma=10.0, kernel=linear, score=0.936, total=   0.2s
[CV] C=46.41588833612777, gamma=10.0, kernel=linear ..................
[CV]  C=46.41588833612777, gamma

[CV]  C=100.0, gamma=3.593813663804626, kernel=linear, score=0.925, total=   0.2s
[CV] C=100.0, gamma=3.593813663804626, kernel=linear .................
[CV]  C=100.0, gamma=3.593813663804626, kernel=linear, score=0.934, total=   0.2s
[CV] C=100.0, gamma=3.593813663804626, kernel=rbf ....................
[CV]  C=100.0, gamma=3.593813663804626, kernel=rbf, score=0.850, total=   0.5s
[CV] C=100.0, gamma=3.593813663804626, kernel=rbf ....................
[CV]  C=100.0, gamma=3.593813663804626, kernel=rbf, score=0.850, total=   0.5s
[CV] C=100.0, gamma=3.593813663804626, kernel=rbf ....................
[CV]  C=100.0, gamma=3.593813663804626, kernel=rbf, score=0.845, total=   0.5s
[CV] C=100.0, gamma=5.994842503189409, kernel=linear .................
[CV]  C=100.0, gamma=5.994842503189409, kernel=linear, score=0.936, total=   0.2s
[CV] C=100.0, gamma=5.994842503189409, kernel=linear .................
[CV]  C=100.0, gamma=5.994842503189409, kernel=linear, score=0.925, total=   0.2s
[CV] C=10

[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  3.8min finished


CPU times: user 3min 42s, sys: 1.39 s, total: 3min 44s
Wall time: 3min 46s


GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': array([  0.1       ,   0.21544347,   0.46415888,   1.        ,
         2.15443469,   4.64158883,  10.        ,  21.5443469 ,
        46.41588834, 100.        ]),
                         'gamma': array([ 0.1       ,  0.16681005,  0.27825594,  0.46415888,  0.77426368,
        1.29154967,  2.15443469,  3.59381366,  5.9948425 , 10.        ]),
                         'kernel': ['linear', 'rbf']},
             verbose=3)

In [86]:
# print best parameters and cross validation score
print(grid_search.best_score_)
print(grid_search.best_params_)

0.9506055342327069
{'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}


In [88]:
# evaluate metrics on the test set
y_pred = grid_search.predict(X_test_transformed)

print("\nPrecision: {:.2f}%".format(
    100 * precision_score(y_test, y_pred)))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_pred)))


Precision: 98.26%
Recall: 97.92%


The evaluated metrics on the test set using the grid search method on svm classifier were comparable to the logistic regression classifier. Both pipelines gave precision and recall scores above 97.5%.